In [20]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

In [21]:
env = ".env"
load_dotenv(env)

True

In [22]:
API_KEY = os.getenv('CHATPDF_KEY')
API_URL_UPLOAD = "https://api.chatpdf.com/v1/sources/add-file"
API_URL_CHAT = "https://api.chatpdf.com/v1/chats/message"

In [23]:
def enviarPDF(arquivo):
    files = [
        ('file', ('file', open(arquivo, 'rb'), 'application/octet-stream'))
    ]
    headers = {
        'x-api-key': API_KEY
    }

    response = requests.post(
        API_URL_UPLOAD, headers=headers, files=files)

    if response.status_code == 200:
        SRC_ID = response.json()['sourceId']
        # print('ARQUIVO ENVIADO COM SUCESSO!')
        # print('SOURCE_ID:', SRC_ID)
        return SRC_ID
    else:
        print('Status:', response.status_code)
        print('Error:', response.text)

In [24]:
def perguntar(pergunta1, src_id):
    headers = {
        'x-api-key': API_KEY,
        "Content-Type": "application/json",
    }

    data = {
        'sourceId': src_id,
        'messages': [
            {
                'role': "user",
                'content': pergunta1,
            }
        ]
    }

    response = requests.post(
        API_URL_CHAT, headers=headers, json=data)

    if response.status_code == 200:
        RESPOSTA = response.json()['content']
        # print('Result:', RESPOSTA)
        return RESPOSTA
    else:
        print('Status:', response.status_code)
        print('Error:', response.text)


In [27]:
upload_id = enviarPDF('./pdf/cursobasicodevinho.pdf')
resposta = perguntar('conte as paginas e leia a pagina de numero 13, eu tenho uma tabela extraia essa tabela e me responda no formato de csv sem texto extra',  upload_id)
print(resposta)

import pandas as pd
import tabula
documento = tabula.convert_into('./pdf/cursobasicodevinho.pdf', 'out.csv', output_format='csv', pages="13")
dataframe = pd.read_csv('out.csv')
dataframe.head()


Não foi mencionada uma página 13 no conteúdo fornecido. Como não encontrei a tabela mencionada, não posso extrair informações específicas dela. Se tiver alguma outra pergunta ou precisar de ajuda com algo mais, por favor, me avise.


,Unnamed: 0,Unnamed: 1,EXAME VISUAL (COR)
0,vinho,variação,descrição
1,NaN,Incolor,"Com pouca ou nenhuma cor, similar à água."
2,NaN,Verdoso,Amarelo-claro com reflexos verdes.
3,NaN,Palha,Amarelo-claro com reflexos palha.
4,Branco,Amarelo,"Amarelo definido, sem reflexos."


In [41]:
from pdf2docx import Converter
from docx import Document

# arquivo = Converter('./pdf/cursobasicodevinho.pdf')
# arquivo.convert('./textos/document.docx')
# arquivo.close()

def extrair_tabelas(nome_arquivo):
    doc = Document(nome_arquivo)
    tabelas = []
    for tabela in doc.tables:
        dados_tabela = []
        for linha in tabela.rows:
            dados_linha = []
            for célula in linha.cells:
                dados_linha.append(célula.text)
            dados_tabela.append(dados_linha)
        tabelas.append(dados_tabela)
    return tabelas


tabelas = extrair_tabelas('./textos/document.docx')

# Exibindo os dados extraídos
for num_tabela, tabela in enumerate(tabelas, start=1):
    print(f"Tabela {num_tabela}:")
    for linha in tabela:
        print("\t".join(linha))
    print()
